In [1]:
# Import Libraries
import numpy as np
import re
import pandas as pd
import chess.pgn
import io

In [2]:
games = pd.DataFrame(columns=["White Name", "Black Name", "Opening", "White Elo", "Black Elo", "PGN"])

with open("games_condensed.txt", "r") as file:
    for line in file:
        line = line.split(",")

        # Don't include wacky variants like Chess960
        if "Variant" in line[3]:
            continue
        
        # Remove unnecessary features
        indices=[2, 3, 4, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 19]  # Manually found
        for i in sorted(indices, reverse=True):
            del line[i]

        # Parse strings to get valuable information
        pattern = r'\\"(.*?)\\"'  # This string pattern encloses all important info
        for i in range(0, 5):
            match = re.search(pattern, line[i])
            if match:
                line[i] = match.group(1)
        
        games.loc[games.shape[0]] = line

In [3]:
test_game = games["PGN"][0]
# each game is currently a row. we need some way to split each game into FEN files.

In [4]:
io.StringIO(test_game)
test_game = chess.pgn.read_game(io.StringIO(test_game))

In [5]:
# Iterate through all moves and play them on a board.
fen_lst = []
board = test_game.board()
for move in test_game.mainline_moves():
    board.push(move)
    # print("new move: \n", board)
    fen_lst.append(board.fen())

In [9]:
import asyncio
import chess.engine

async def main() -> None:
    transport, engine = await chess.engine.popen_uci(r"C:\Users\naga2\Downloads\stockfish_15.1_win_x64_avx2\stockfish_15.1_win_x64_avx2\stockfish-windows-2022-x86-64-avx2.exe")
    board = chess.Board(fen_lst[12])
    info = await engine.analyse(board, chess.engine.Limit(depth=20))
    print(info["score"])
    await engine.quit() 
    
asyncio.set_event_loop_policy(chess.engine.EventLoopPolicy())
asyncio.run(main())   

In [ ]:
chess.Board(fen_lst[-1])